In [3]:
import numpy as np
import imutils
import cv2
from matplotlib import pyplot as plt

def Detect_Feature_And_KeyPoints(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect and extract features from the image
    descriptors = cv2.xfeatures2d.SIFT_create()
    (Keypoints, features) = descriptors.detectAndCompute(image, None)

    Keypoints = np.float32([i.pt for i in Keypoints])
    return (Keypoints, features)
    
def get_Allpossible_Match(featuresA,featuresB):

    # compute the all matches using euclidean distance and opencv provide
    #DescriptorMatcher_create() function for that
    match_instance = cv2.DescriptorMatcher_create("BruteForce")
    All_Matches = match_instance.knnMatch(featuresA, featuresB, 2)

    return All_Matches

def All_validmatches(AllMatches,lowe_ratio):
    #to get all valid matches according to lowe concept..
    valid_matches = []

    for val in AllMatches:
        if len(val) == 2 and val[0].distance < val[1].distance * lowe_ratio:
            valid_matches.append((val[0].trainIdx, val[0].queryIdx))

    return valid_matches



def matchKeypoints(KeypointsA, KeypointsB, featuresA, featuresB,lowe_ratio, max_Threshold):

    AllMatches = get_Allpossible_Match(featuresA,featuresB);
    valid_matches = All_validmatches(AllMatches,lowe_ratio)

    if len(valid_matches) > 4:
        # construct the two sets of points
        pointsA = np.float32([KeypointsA[i] for (_,i) in valid_matches])
        pointsB = np.float32([KeypointsB[i] for (i,_) in valid_matches])

        (Homograpgy, status) = cv2.findHomography(pointsA, pointsB, cv2.RANSAC, max_Threshold)

        return (valid_matches, Homograpgy, status)
    else:
        return None

def getwarp_perspective(imageA,imageB,Homography):
    val = imageA.shape[1] + imageB.shape[1]
    result_image = cv2.warpPerspective(imageA, Homography, (val , imageA.shape[0]))

    return result_image

def get_points(imageA,imageB):

    (hA, wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB

    return vis


def draw_Matches(imageA, imageB, KeypointsA, KeypointsB, matches, status):

    (hA,wA) = imageA.shape[:2]
    vis = get_points(imageA,imageB)

    # loop over the matches
    for ((trainIdx, queryIdx), s) in zip(matches, status):
        if s == 1:
            ptA = (int(KeypointsA[queryIdx][0]), int(KeypointsA[queryIdx][1]))
            ptB = (int(KeypointsB[trainIdx][0]) + wA, int(KeypointsB[trainIdx][1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

    return vis


def image_stitch(images, lowe_ratio=0.75, max_Threshold=4.0,match_status=False):

    #detect the features and keypoints from SIFT
    (imageB, imageA) = images
    (KeypointsA, features_of_A) = Detect_Feature_And_KeyPoints(imageA)
    (KeypointsB, features_of_B) = Detect_Feature_And_KeyPoints(imageB)

    #got the valid matched points
    Values = matchKeypoints(KeypointsA, KeypointsB,features_of_A, features_of_B, lowe_ratio, max_Threshold)

    if Values is None:
        return None

    #to get perspective of image using computed homography
    (matches, Homography, status) = Values
    result_image = getwarp_perspective(imageA,imageB,Homography)
    result_image[0:imageB.shape[0], 0:imageB.shape[1]] = imageB

    # check to see if the keypoint matches should be visualized
    if match_status:
        vis = draw_Matches(imageA, imageB, KeypointsA, KeypointsB, matches,status)

        return (result_image, vis)

    return result_image


In [4]:


# "Enter the number of images you want to concantenate:"
no_of_images = 3
# "Enter the image name in order of left to right in way of concantenation:"
filename = ['images/scene1_a.jpeg', 'images/scene1_b.jpeg', 'images/scene1_c.jpeg']

images = []

for i in range(no_of_images):
    images.append(cv2.imread(filename[i]))

#We need to modify the image resolution and keep our aspect ratio use the function imutils

for i in range(no_of_images):
    images[i] = imutils.resize(images[i], width=400)

for i in range(no_of_images):
    images[i] = imutils.resize(images[i], height=400)


if no_of_images==2:
    (result, matched_points) = image_stitch([images[0], images[1]], match_status=True)
else:
    (result, matched_points) = image_stitch([images[no_of_images-2], images[no_of_images-1]], match_status=True)
    for i in range(no_of_images - 2):
        (result, matched_points) = image_stitch([images[no_of_images-i-3],result], match_status=True)



cv2.imwrite("Panorama_image.jpg",result)



True